# Introductions
This code prepares a CSV file that the interface expects. 

1. This will take the series of Markov transition files for each interaction log dataset, and combine them into `temp1_merged_trasnsition data.csv`. 
2. This is then combined with the other metrics already in `datasetAll.csv` - creating `temp2_all_merged_data.csv`. 
3. We make a JSON file of preferences for any new metrics created by modifying the `datasetAll.json`. Note that this is the JSON version used by the visual interface to specify the names, descriptions and how many grid lines are used for each metric.
4. Finally, we drop columns that are not meaningful or worth keeping by listing the available metric names and then dropping them by name. This step makes `temp3_output_csv_file.csv`.

When scripts are finished running be sure to rename the output file `temp3_output_csv_file.csv` to `datasetAll.csv`!

## 1. Merge the Transition probability data
The set of transition probability files are CSV files that all have the same header, and just need to be combined into a single file. The files essentially are the likelihood that a user will conduct a specific event type. Essentially a table of Markov probabilities where each row is a user interaction session and the columns are the types of interaction transitions.

In [2]:
import pandas as pd

# List of CSV file names to merge
csv_files = [  # Replace with your file names
    "markov_1.csv",
    "markov_2.csv",
    "markov_3.csv",
    "markov_4.csv",
    "markov_5.csv",
]

# Name of the common column used for merging
merge_column_name = "FileName"  # Replace with the desired column name

# Initialize an empty DataFrame as the base
merged_data = pd.DataFrame()

# Loop through each CSV file and merge
for file in csv_files:
    df = pd.read_csv(file)

    # If it's the first file, set it as the base DataFrame
    if merged_data.empty:
        merged_data = df
    else:
        # Merge the current DataFrame with the base DataFrame based on the common column
        merged_data = merged_data.merge(df, how="outer")

# Fill missing values with appropriate values (e.g., NaN or a specific value)
merged_data = merged_data.fillna("")

# Save the merged data to a new CSV file
merged_data.to_csv("temp1_merged_transitions_data.csv", index=False)

# Display the merged data
merged_data.head()

,FileName,DataExplorationCounts,VisualExplorationCounts,InsightActionCounts,TheorizingCounts,DiscoveringCounts,AuditingCounts,OrganizingCounts,RecognizingCounts,TrackingCounts,...,DataExploringUnusual,VisualExploringUnusual,InsightActionUnusual,TheorizingUnusual,DiscoveringUnusual,AuditingUnusual,OrganizingUnusual,RecognizingUnusual,TrackingUnusual,TotalUnusual
0,Arms_P1_InteractionsLogs.json,204.0,144.0,49.0,101.0,80.0,67.0,67.0,52.0,114.0,...,0.254902,0.000000,0.306122,0.495050,0.450000,0.477612,0.134328,0.5,0.263158,0.284738
1,Arms_P2_InteractionsLogs.json,192.0,255.0,15.0,115.0,50.0,41.0,37.0,27.0,124.0,...,0.114583,0.000000,0.866667,0.043478,0.380000,0.317073,0.081081,1.0,0.362903,0.171729
2,Arms_P3_InteractionsLogs.json,468.0,163.0,29.0,186.0,68.0,80.0,24.0,33.0,175.0,...,0.170940,0.000000,0.241379,0.327957,1.000000,0.162500,0.458333,1.0,0.228571,0.255302
3,Arms_P4_InteractionsLogs.json,271.0,196.0,32.0,134.0,167.0,161.0,51.0,46.0,139.0,...,0.147601,0.000000,0.375000,0.305970,0.353293,0.198758,0.254902,1.0,0.122302,0.217210
4,Arms_P5_InteractionsLogs.json,310.0,205.0,2.0,149.0,29.0,19.0,48.0,37.0,160.0,...,0.141935,0.117073,1.000000,0.194631,1.000000,1.000000,0.000000,1.0,0.162500,0.218978


# 2. Merge the unusual Transition types with the rest of the metrics

In [3]:
import pandas as pd

# List of CSV file names to merge
csv_files = [
    "./datasetAll_231117.csv",
    "temp1_merged_transitions_data.csv",
]  # Replace with your 2 file names

# Name of the common column used for merging
left_column_name = "session"  # Replace with the desired column names
right_column_name = "FileName"

# Initialize an empty DataFrame as the base
merged_data = pd.DataFrame()

# Loop through each CSV file and merge
for file in csv_files:
    df = pd.read_csv(file)

    # If it's the first file, set it as the base DataFrame
    if merged_data.empty:
        merged_data = df
    else:
        # Merge the current DataFrame with the base DataFrame based on the common column
        merged_data = merged_data.merge(
            df, left_on=left_column_name, right_on=right_column_name, how="outer"
        )
        merged_data.drop(
            right_column_name, axis=1, inplace=True
        )  # Remove right column since we have merged this.

# Fill missing values with appropriate values (e.g., NaN or a specific value)
merged_data = merged_data.fillna("0")

# Save the merged data to a new CSV file
merged_data.to_csv("temp2_all_merged_data.csv", index=False)

# Display the merged data
merged_data.head()

,session,section,total_think_aloud_count,total_search_count,total_topic_change_count,total_mouse_hover_count,total_draging_count,total_open-doc_count,total_reading_count,total_highlight_count,...,DataExploringUnusual,VisualExploringUnusual,InsightActionUnusual,TheorizingUnusual,DiscoveringUnusual,AuditingUnusual,OrganizingUnusual,RecognizingUnusual,TrackingUnusual,TotalUnusual
0,Arms_P1_InteractionsLogs.json,Arms Dealing,137,82,10,143,169,134,159,21,...,0.254902,0.000000,0.306122,0.495050,0.450000,0.477612,0.134328,0.5,0.263158,0.284738
1,Arms_P2_InteractionsLogs.json,Arms Dealing,69,27,6,275,132,91,233,8,...,0.114583,0.000000,0.866667,0.043478,0.380000,0.317073,0.081081,1.0,0.362903,0.171729
2,Arms_P3_InteractionsLogs.json,Arms Dealing,78,46,15,167,207,140,522,7,...,0.170940,0.000000,0.241379,0.327957,1.000000,0.162500,0.458333,1.0,0.228571,0.255302
3,Arms_P4_InteractionsLogs.json,Arms Dealing,74,76,10,152,229,149,337,141,...,0.147601,0.000000,0.375000,0.305970,0.353293,0.198758,0.254902,1.0,0.122302,0.217210
4,Arms_P5_InteractionsLogs.json,Arms Dealing,32,66,11,264,139,239,173,1,...,0.141935,0.117073,1.000000,0.194631,1.000000,1.000000,0.000000,1.0,0.162500,0.218978


## 3. Modify or create a Config File to include any newly merged column names

In [4]:
import csv
import json

def csv_to_json(input_csv_file, input_json_file, output_json_file):
    # Initialize an empty dictionary to store the JSON data
    json_data = {}

    # Read the existing JSON data from the output file if it exists
    try:
        with open(input_json_file, 'r') as json_file:
            json_data = json.load(json_file)
    except FileNotFoundError:
        pass

    # Read the CSV file
    with open(input_csv_file, 'r') as csv_file:
        csv_reader = csv.DictReader(csv_file)

        # Iterate through the columns in the CSV file
        for column_name in csv_reader.fieldnames:
            # Check if the column name already exists in the JSON data
            if column_name not in json_data:
                json_data[f'{column_name}'] = {
                    "name": column_name,
                    "unit": "",
                    "description": f"a_written_description for {column_name}",
                    "majorTicks": 4,
                    "minorTicks": 1,
                    "places": 0
                }

    # Write the JSON data to the output file
    with open(output_json_file, 'w') as json_file:
        json.dump(json_data, json_file, indent=4)

# Example usage:
input_csv_file = 'temp2_all_merged_data.csv'
input_json_file = 'datasetAll.json'
output_json_file = input_json_file
csv_to_json(input_csv_file, input_json_file, output_json_file)


# 4. Drop columns with limited meaning.

In [5]:
if 'pd' not in globals():
    import pandas as pd

# get list of column names
def get_column_names_from_csv(csv_file_path):
    try:
        # Read the CSV file into a DataFrame
        df = pd.read_csv(csv_file_path)

        # Get the column names as a list
        column_names = df.columns.tolist()

        return column_names
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


# Example usage:
csv_file_path = "temp2_all_merged_data.csv"  # Replace with the path to your CSV file
column_names = get_column_names_from_csv(csv_file_path)

if column_names is not None:
    print(len(column_names), "Column names in the CSV file:")
    for name in column_names:
        print(f"\"{name}\",")

61 Column names in the CSV file:
"session",
"section",
"total_think_aloud_count",
"total_search_count",
"total_topic_change_count",
"total_mouse_hover_count",
"total_draging_count",
"total_open-doc_count",
"total_reading_count",
"total_highlight_count",
"total_connection_count",
"total_create_note_count",
"total_add_note_count",
"think_aloud_ratio",
"search_ratio",
"topic_change_ratio",
"mouse_hover_ratio",
"draging_ratio",
"open-doc_ratio",
"reading_ratio",
"highlight_ratio",
"connection_ratio",
"create_note_ratio",
"add_note_ratio",
"total_interaction_count",
"total_duration",
"repeat_searches",
"prop_repeat_searches",
"search_term_similarity",
"search_time_std_dev",
"search_open_overlap",
"search_term_efficiency",
"DataExplorationCounts",
"VisualExplorationCounts",
"InsightActionCounts",
"TheorizingCounts",
"DiscoveringCounts",
"AuditingCounts",
"OrganizingCounts",
"RecognizingCounts",
"TrackingCounts",
"DataExploringUsual",
"VisualExploringUsual",
"InsightActionUsual",
"TheorizingU

In [8]:
if "pd" not in globals():
    import pandas as pd

# Define the list of columns you want to remove
columns_to_remove = [
    "total_topic_change_count",
    "total_mouse_hover_count",
    "total_draging_count",
    "total_reading_count",
    "total_highlight_count",
    "total_connection_count",
    "total_create_note_count",
    "total_add_note_count",
    "topic_change_ratio",
    "mouse_hover_ratio",
    "draging_ratio",
    "reading_ratio",
    "highlight_ratio",
    "connection_ratio",
    "create_note_ratio",
    "add_note_ratio",
    "total_initialized_count",
    "total_query_results_count",
    "total_read_count",
    "total_playaudiofileinnewwindow_count",
    "total_collapse_count",
    "total_find_count",
    "total_dispositionupdated_count",
    "total_noteupdated_count",
    "total_listening_count",
    "total_createdocumentbucket_count",
    "total_addtodocumentbucket_count",
    "total_removefromdocumentbucket_count",
    "total_tabs_to_questions_count",
    "total_tabs_to_cap_count",
    "total_edit_document_bucket_count",
    "total_loaddocumentlist_count",
    "total_show_count",
    "total_getdocuments_count",
    "total_answers_question_count",
    "total_tabs_to_supplementary_count",
    "total_tabs_to_bundle_count",
    "total_reads_count",
    "total_tabs_to_abreviations_count",
    "total_answers_confidence_count",
    "total_tabs_to_bundles_count",
    "total_answer_count",
    "total_initiate_count",
    "total_custom_count",
    "initialized_ratio",
    "query_results_ratio",
    "read_ratio",
    "playaudiofileinnewwindow_ratio",
    "collapse_ratio",
    "find_ratio",
    "dispositionupdated_ratio",
    "noteupdated_ratio",
    "listening_ratio",
    "createdocumentbucket_ratio",
    "addtodocumentbucket_ratio",
    "removefromdocumentbucket_ratio",
    "tabs_to_questions_ratio",
    "tabs_to_cap_ratio",
    "edit_document_bucket_ratio",
    "loaddocumentlist_ratio",
    "show_ratio",
    "getdocuments_ratio",
    "answers_question_ratio",
    "tabs_to_supplementary_ratio",
    "tabs_to_bundle_ratio",
    "reads_ratio",
    "tabs_to_abreviations_ratio",
    "answers_confidence_ratio",
    "tabs_to_bundles_ratio",
    "answer_ratio",
    "initiate_ratio",
    "custom_ratio",
    "total_switch_count",
    "total_discover_count",
    "total_save_count",
    "total_revisit_count",
    "total_assign_count",
    "switch_ratio",
    "discover_ratio",
    "save_ratio",
    "revisit_ratio",
    "assign_ratio",
]

# Load the CSV file into a DataFrame
file_path = csv_file_path
df = pd.read_csv(file_path)

# Create a list to store columns that don't exist in the DataFrame
columns_not_found = []

print(f"Attempting to remove {len(columns_to_remove)} colums")
# Remove the specified columns (if they exist)
for column in columns_to_remove:
    if column in df.columns:
        df.drop(columns=column, inplace=True)
    else:
        columns_not_found.append(column)

# Save the modified DataFrame back to a new CSV file
output_file_path = "temp3_output_csv_file.csv"
df.to_csv(output_file_path, index=False)


if columns_not_found:
    print(
        len(columns_not_found),
        "columns not found in the input file. \nSkipped the following:",
        columns_not_found,
    )

print("Success, output dimentions of the dataset:", df.shape)
print(f"output to {output_file_path}. Don't forget to rename to 'datasetAll.csv'")

Attempting to remove 82 colums
66 columns not found in the input file. 
Skipped the following: ['total_initialized_count', 'total_query_results_count', 'total_read_count', 'total_playaudiofileinnewwindow_count', 'total_collapse_count', 'total_find_count', 'total_dispositionupdated_count', 'total_noteupdated_count', 'total_listening_count', 'total_createdocumentbucket_count', 'total_addtodocumentbucket_count', 'total_removefromdocumentbucket_count', 'total_tabs_to_questions_count', 'total_tabs_to_cap_count', 'total_edit_document_bucket_count', 'total_loaddocumentlist_count', 'total_show_count', 'total_getdocuments_count', 'total_answers_question_count', 'total_tabs_to_supplementary_count', 'total_tabs_to_bundle_count', 'total_reads_count', 'total_tabs_to_abreviations_count', 'total_answers_confidence_count', 'total_tabs_to_bundles_count', 'total_answer_count', 'total_initiate_count', 'total_custom_count', 'initialized_ratio', 'query_results_ratio', 'read_ratio', 'playaudiofileinnewwindo

Expected Size is 46 rows x 45 columns at last change (12/03/2023)